In [ ]:
from google import genai
from google.genai import types

In [ ]:
import pika;
import json;
import mysql.connector;

In [ ]:
text_input = '''President Trump on Thursday moved forward with new rules aimed at undermining China’s strength in commercial shipping, but the measures were softened after ocean carriers and importers said the regulations could push up the cost of shipping.Chinese shipping companies and many owners of Chinese-built ships will now have to pay fees when they dock at U.S. ports. But under the more lenient version of the rules, the biggest shipping companies will most likely pay significantly less and smaller ones will be exempt.The Trump administration said the measures were necessary because China had used unfair trade practices like subsidies to become dominant in shipbuilding. The rules also aim to foster the growth of the American shipbuilding industry, which has withered in recent decades. The rules give shipping lines refunds on their fees if they buy American-made ships in the next few years.“The Trump administration’s actions will begin to reverse Chinese dominance, address threats to the U.S. supply chain and send a demand signal for U.S.-built ships,” Jamieson Greer, the head of the Office of the United States Trade Representative, which formulated the rules, said in statement.The new rules originated from a petition filed during the Biden administration by a collection of unions, including the United Steelworkers and the AFL-CIO, that requested an investigation into Chinese shipbuilding. The Biden administration carried out the investigation and issued a finding shortly before Mr. Trump took office in January. That finding said China had displaced foreign firms in the shipbuilding sector and unfairly hurt U.S. commerce.David McCall, the president of the United Steelworkers, said the government’s “thorough investigation validated our charges, and today’s announcement lays out a series of strong steps to restore U.S. shipbuilding capacity.”Both Democrats and Republicans have grown more concerned about the nation’s heavy reliance on China to move goods around the world. But isolating and penalizing Chinese shipping could strain supply chains. The fleets of the biggest shipping lines contain large numbers of Chinese-built vessels, and Chinese-owned operators transport huge amounts of goods to the United States. Critics of the new rules say they will only add costs to supply chains at a time when importers have to bear the expense of new tariffs imposed by the Trump administration.“When ocean carriers raise rates, American families will pay the price through higher costs and growing product shortages, at a time when they can least afford it,” said Nate Herman, a senior vice president at the American Apparel and Footwear Association.Skeptics in the maritime industry said the new rules would not lead to a renaissance in U.S. shipbuilding because American shipyards lacked capacity and charged much more for their vessels than foreign rivals did.“I don’t see anything in here that would boost shipbuilding in the U.S.,” said Lars Jensen, the chief executive of Vespucci Maritime, a container shipping consultancy based in Copenhagen.A 2023 report for Congress said China was building hundreds of large ships a year while the United States was building “five or fewer.” After China, South Korea and Japan make most of the large commercial vessels.In making the rules more lenient, the Trump administration did away with a draft provision that would have applied a fee on all ships belonging to a shipping line with a fleet in which 25 percent or more of the vessels were Chinese-built.It also decided against applying a flat $1.5 million fee on Chinese-built vessels. Now, the fee on such vessels will be calculated on weight or number of containers, charging whichever is highest. Starting in October, when the rules take effect, the United States will charge a shipping line $150 per container, rising to $250 in 2028. At $150, a vessel bringing in 7,000 containers would pay just over $1 million.Vessels belonging to Chinese shipping lines will pay a fee based on their weight — $50 per net ton, starting in October, rising to $140 per net ton in 2028. The vessels of Chinese-owned shipping lines will not be charged more than five times a year.Vessels making voyages under 2,000 nautical miles to American ports do not have to pay the fee, a huge relief to the shipping lines that make voyages to the Caribbean and in the Great Lakes. Smaller vessels are also exempt.The original rules suggested that the fees would be charged each time a Chinese-owned or Chinese-built vessel docked at a U.S. port (shipping lines may visit several East Coast ports on one voyage), which prompted fear that ocean carriers would stop visiting smaller ports.But the final regulations say the fee will be charged only at one port.Mr. Jensen, the maritime consultant, said he expected the large shipping lines to rejig their operations to reduce the number of Chinese-built ships sailing to the United States. “The only name of the game right now is how much you minimize the costs by changing where you deploy which ships,” he said.The rules also try to create a path to building vessels in the United States that carry liquefied natural gas, or L.N.G., one of the country’s most important exports. The rules say that, by 2029, 1 percent of L.N.G. exports must be carried on American-built vessels.Colin Grabow, an associate director at the Cato Institute, a research organization that favors free markets, said that the United States had not built an L.N.G. carrier since 1980, and that it would not be able to build one within four years, given the lack of shipyard capacity and expertise in the country.“Getting even one L.N.G. carrier in operation by 2029 is not at all feasible,” he said.'''

In [ ]:
gemini_key = ''
with open("config", "r", encoding="utf-8") as f:
    gemini_key = f.read()

In [ ]:
mq_host = "192.168.0.202" #'rabbitmq'
mq_port = 5672
mysql_host = "192.168.0.202" #'news-mysql'
mysql_port = 4000 #3306

In [ ]:
mq_conn = pika.BlockingConnection(pika.ConnectionParameters(host=mq_host, port=mq_port))
channel = mq_conn.channel()

db_conn = mysql.connector.connect(
    host=mysql_host,
    port=mysql_port,
    user="root",
    password="password",
    database="news"
)

In [ ]:
test_public = '''{"id": "https://www.nytimes.com/2025/04/20/world/asia/australia-waves-queensland-sydney.html", "title": "6 Killed as Giant Waves Batter Australia", "url": "https://www.nytimes.com/2025/04/20/world/asia/australia-waves-queensland-sydney.html", "author": "Yan Zhuang", "date": "Sun, 20 Apr 2025 09:37:40 +0000"}'''

In [ ]:
channel.basic_publish(exchange='news.fetch', routing_key='summary', body='test')

In [ ]:
method_frame, header_frame, body = channel.basic_get(queue='summary', auto_ack=False)
channel.basic_ack(delivery_tag=method_frame.delivery_tag)

In [ ]:
get_sql = "SELECT context FROM news WHERE URL = %s"
update_sql = "UPDATE news SET summary = %s WHERE URL = %s;"

In [ ]:
test_URL = 'https://www.nytimes.com/2025/04/20/briefing/ukraine-easter-truce-syria-massacre-state-department-africa.html'

In [ ]:
cursor = db_conn.cursor(dictionary=True)
cursor.execute(get_sql,[test_URL])
rows = cursor.fetchall()

In [ ]:
rows

In [ ]:
text_input=rows[0]['context']

In [ ]:
client = genai.Client(api_key=gemini_key)

response = client.models.generate_content(
  model="gemini-2.0-flash", 
  config=types.GenerateContentConfig(
    system_instruction=(
      "You are an expert summarizer. Your task is to summarize long texts into a clear, informative summary "
      "written in well-structured paragraphs. The summary should be less than 300 words. "
      "Do not use bullet points or numbered lists. "
      "Focus on capturing the key points, main arguments, and important details. "
      "Avoid unnecessary repetition and do not include personal opinions."
    )
  ),
  contents=text_input
)
print(response.text)

In [ ]:
response.text

In [ ]:
cursor = db_conn.cursor()
cursor.execute(update_sql,(response.text,test_URL))
db_conn.commit()
